<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

In [2]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [3]:
# enddate = date.today()
# startdate = enddate - relativedelta(years=4)

# df = yf.download('HINDZINC.NS', start=startdate, end=enddate)


In [4]:
#ticker_yahoo = yf.Ticker('HINDZINC.NS')
# last_quote = ticker_yahoo.fast_info['last_price']
# MA200 = ticker_yahoo.fast_info['two_hundred_day_average']
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)
#print(tickers)

In [5]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [6]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')
  


def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    #print(f"{ticker} {info['last_price']} {info['year_high']}")
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ath_list))
    return results

results = paralel_func(ath_list)


In [7]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)
#ticker_list = ticker_list.loc[(ticker_list['Last_price'] < ticker_list['ATH'] * 0.95) & (ticker_list['Last_price'] > ticker_list['ATH'] * 0.9) & (ticker_list['Last_price'] > ticker_list['two_EMA'])]

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)
#ticker_list
ticker_list.to_csv('Watchlist.csv', index=False)


In [8]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today()

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]
#tickers
df = yf.download(tickers, start= startdate, end= enddate)


[*********************100%***********************]  101 of 101 completed


In [9]:
# df.ta.supertrend(high='High', low='Low', close='Close', length=10, multiplier=1, append=True)
# df.to_csv('data1.csv')
#tickers
#help(ta.supertrend)

In [16]:
pd.set_option('display.max_rows', 10)

ax_df = df.copy()

# ax_df = ta.supertrend(close=df[('Close', 'ABBOTINDIA.NS')]
#                                                          , high=df[('High', 'ABBOTINDIA.NS')]
#                                                          , low=df[('Low', 'ABBOTINDIA.NS')]
#                                                          , length=10, multiplier=1)

# df_copy['SUPERTd_10_1', 'ABBOTINDIA.NS'] = ax_df['SUPERTd_10_1.0']
# df_copy

for ticker in tickers:
  #df[('EMA_200',ticker)] = ta.ema(df.loc[:,('Close', ticker)], length=200)
  SUPERT_10_1 =  ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)
  SUPERT_11_2 =  ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)
  SUPERT_12_3 =  ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)
  ax_df[('SUPERTd_10_1',ticker)] = SUPERT_10_1['SUPERTd_10_1.0']
  ax_df[('SUPERTd_11_2',ticker)] = SUPERT_11_2['SUPERTd_11_2.0']
  ax_df[('SUPERTd_12_3',ticker)] = SUPERT_12_3['SUPERTd_12_3.0']

   
ax_df

<ipython-input-16-e19edffbe003>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ax_df[('SUPERTd_12_3',ticker)] = SUPERT_12_3['SUPERTd_12_3.0']
<ipython-input-16-e19edffbe003>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ax_df[('SUPERTd_10_1',ticker)] = SUPERT_10_1['SUPERTd_10_1.0']
<ipython-input-16-e19edffbe003>:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1

Adj Close                                        \
                          ABBOTINDIA.NS      ACE.NS AEGISCHEM.NS AKZOINDIA.NS   
Date                                                                            
2019-02-05 00:00:00+05:30   7865.229004   82.149124   190.172180  1648.566284   
2019-02-06 00:00:00+05:30   7886.305664   81.757942   193.560333  1617.627319   
2019-02-07 00:00:00+05:30   8002.127930   83.567177   192.398697  1613.220947   
2019-02-08 00:00:00+05:30   7810.838867   82.735909   186.300018  1612.564819   
2019-02-11 00:00:00+05:30   7446.035645   82.002426   185.622391  1594.564087   
...                                 ...         ...          ...          ...   
2023-01-30 00:00:00+05:30  20985.699219  322.100006   373.600006  2242.199951   
2023-01-31 00:00:00+05:30  20817.050781  332.899994   381.649994  2244.899902   
2023-02-01 00:00:00+05:30  20700.449219  323.750000   360.100006  2231.050049   
2023-02-02 00:00:00+05:30  20480.300781  331.399994   358.649994  2213.899902   
2023-02-03 00:00:00+05:30  20872.250000  329.549988   367.200012  2210.300049   

                                                                              \
                          AMARAJABAT.NS  APARINDS.NS  APTECHT.NS ASHOKLEY.NS   
Date                                                                           
2019-02-05 00:00:00+05:30    735.662170   566.409790  153.025513   74.353378   
2019-02-06 00:00:00+05:30    747.736389   578.347534  151.231689   74.715416   
2019-02-07 00:00:00+05:30    747.354553   587.548645  150.679764   79.421898   
2019-02-08 00:00:00+05:30    718.815735   580.754028  149.529892   78.788338   
2019-02-11 00:00:00+05:30    706.741577   571.128296  144.056473   75.213219   
...                                 ...          ...         ...         ...   
2023-01-30 00:00:00+05:30    569.950012  1405.699951  337.049988  144.300003   
2023-01-31 00:00:00+05:30    566.549988  1686.800049  353.350006  149.500000   
2023-02-01 00:00:00+05:30    593.700012  1946.150024  343.649994  147.699997   
2023-02-02 00:00:00+05:30    615.549988  2015.949951  357.899994  152.050003   
2023-02-03 00:00:00+05:30    605.049988  1931.099976  362.500000  153.899994   

                                                    ...  SUPERTd_12_3  \
                           ASTRAZEN.NS AXISBANK.NS  ... UNIVCABLES.NS   
Date                                                ...                 
2019-02-05 00:00:00+05:30  1586.670288  728.595154  ...             1   
2019-02-06 00:00:00+05:30  1605.585938  724.849792  ...             1   
2019-02-07 00:00:00+05:30  1791.448486  729.244385  ...             1   
2019-02-08 00:00:00+05:30  1806.022217  718.507690  ...             1   
2019-02-11 00:00:00+05:30  1834.121094  709.968262  ...             1   
...                                ...         ...  ...           ...   
2023-01-30 00:00:00+05:30  3357.350098  871.849976  ...             1   
2023-01-31 00:00:00+05:30  3361.050049  871.599976  ...             1   
2023-02-01 00:00:00+05:30  3386.699951  857.250000  ...             1   
2023-02-02 00:00:00+05:30  3367.500000  868.450012  ...             1   
2023-02-03 00:00:00+05:30  3304.050049  882.599976  ...             1   

                           SUPERTd_10_1  SUPERTd_11_2  SUPERTd_12_3  \
                          VENUSPIPES.NS VENUSPIPES.NS VENUSPIPES.NS   
Date                                                                  
2019-02-05 00:00:00+05:30             1             1             1   
2019-02-06 00:00:00+05:30             1             1             1   
2019-02-07 00:00:00+05:30             1             1             1   
2019-02-08 00:00:00+05:30             1             1             1   
2019-02-11 00:00:00+05:30             1             1             1   
...                                 ...           ...           ...   
2023-01-30 00:00:00+05:30            -1            -1             1   
2023-01-31 00:00:00+05:30            -1  